In [3]:
import time
import numpy as np
import matplotlib as mpl
import matplotlib.colors as cm
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from unyt import yr, Myr, kpc, arcsec, nJy, Mpc, Msun, erg, s, Hz
from astropy.cosmology import Planck18 as cosmo
from scipy import signal
import numpy as np

from synthesizer.grid import Grid
from synthesizer.filters import FilterCollection as Filters
from synthesizer.load_data.load_camels import load_CAMELS_IllustrisTNG
from synthesizer.kernel_functions import Kernel

In [4]:
# Define the grid
grid_name = "bc03-2016-Miles_chabrier-0.1,100.hdf5"
grid_dir = "/Users/am/Documents/CAMELS/data/grids/"
grid = Grid(grid_name, grid_dir=grid_dir, read_lines=False)
print(grid)

------------------------------
SUMMARY OF GRID
log10age: [ 5.          5.          5.14998846  5.20000187  5.25000503  5.3000082
  5.3499959   5.40000235  5.45000263  5.50000307  5.55000809  5.60000309
  5.65000623  5.7000024   5.74999898  5.80000183  5.85000259  5.90000097
  5.95000442  6.          6.01998816  6.04000864  6.06001754  6.08001285
  6.09999123  6.12001425  6.14000495  6.15998805  6.18001112  6.20000187
  6.22000343  6.23999979  6.2599998   6.28000895  6.3000082   6.3200008
  6.34000762  6.36000613  6.37999404  6.40000235  6.42000528  6.43999547
  6.45999526  6.48000694  6.50000307  6.51999853  6.54000394  6.56000233
  6.58000069  6.60000309  6.62000064  6.64000415  6.66000172  6.67999992
  6.7000024   6.7200021   6.73999931  6.76000005  6.7800003   6.80000183
  6.81999773  6.83999939  6.86000242  6.88000139  6.90000097  6.91999803
  6.93999821  6.9599996   6.97999882  7.          7.01998816  7.04000864
  7.06001754  7.08001285  7.09999123  7.11998131  7.14000495  7.15998

In [5]:
galaxy_start = time.time()

# Create galaxy object
gal_0 = load_CAMELS_IllustrisTNG(
    "../../synthesizer/tests/data/",
    snap_name="snap_000.hdf5",
    fof_name="fof_subhalo_tab_000.hdf5",
)[0]

print("Galaxy created, took:", time.time() - galaxy_start)

print(f"Galaxy has {gal_0.stars.nstars} stellar particles")
print(f"Galaxy gas {gal_0.gas.nparticles} gas particles")

/Users/am/anaconda3/envs/allyenv/lib/python3.9/site-packages/unyt/array.py:1938: RuntimeWarning: invalid value encountered in divide
  out_arr = func(


Galaxy created, took: 6.213754892349243
Galaxy has 431 stellar particles
Galaxy gas 3182 gas particles


In [6]:
galaxy_start = time.time()

# Create galaxy object
gal = load_CAMELS_IllustrisTNG(
    "../../synthesizer/tests/data/",
    snap_name="camels_snap.hdf5",
    fof_name="camels_subhalo.hdf5",
    physical=True,
)[0]

print("Galaxy created, took:", time.time() - galaxy_start)

print(f"Galaxy has {gal.stars.nstars} stellar particles")
print(f"Galaxy gas {gal.gas.nparticles} gas particles")

Galaxy created, took: 0.01028299331665039
Galaxy has 473 stellar particles
Galaxy gas 0 gas particles


# Getting the spectra
To make an image we need to map the stellar particle properties onto the SPS grid defined in the grid object. To do this we use the galaxy’s in built generate_particle_spectra method and create "total" SEDs with both stellar and nebular contributions. This returns an SED object containing lots of helper methods for working with spectra.

Here we will use get_fnu to convert the rest frame SED into an observed SED which takes into account the previously set redshift stored on the stars object. By passing igm=None we assume the default igm contribution (Inoue14). This can be changed to False for no IGM or to another IGM model defined in synthesizer.igm.

In [7]:
spectra_start = time.time()

# Calculate the stellar rest frame SEDs for all particles in erg / s / Hz
sed = gal.stars.get_particle_spectra_incident(grid)

# Calculate the observed SED in nJy
sed.get_fnu(cosmo, gal.redshift, igm=None)

print("Spectra created, took:", time.time() - spectra_start)
print(sed)

Spectra created, took: 0.1226191520690918
----------
SUMMARY OF SED 
Number of wavelength points: 13216 
Wavelength range: [5.60 Å,             360000000.00 Å] 
log10(Peak luminosity/erg/(Hz*s)):             26.50 
log10(Bolometric luminosity/erg/s):[40.4594477  40.99248273 40.76127067 40.8318266  40.94759008 40.53416884
 40.67894554 40.78086144 40.862288   40.56260494 40.60275233 40.78193409
 40.80163803 40.56721204 40.80531832 40.57472902 40.90476185 40.53656248
 40.82247317 40.55524706 40.68158581 40.66739791 40.74831192 40.50958571
 40.72737027 40.36182259 40.68774137 40.53618364 40.640348   40.46964073
 41.07619013 40.36241711 40.7459713  40.65273854 40.91468444 40.49801701
 40.82758002 40.92998481 40.76175843 40.66349269 40.63108744 40.51220094
 40.51788089 40.45822936 40.93599284 40.64825024 40.4882154  40.87117428
 40.58202738 40.60409246 40.71376104 40.61254295 40.61841964 40.852993
 40.83391126 40.76430266 40.51284461 40.59416502 40.41760936 40.75612302
 40.77205624 40.608834

# Defining filters
Next we need to define a set of filters we want photometric images for. This can be done in a number of ways using a FilterCollection (see the filter docs for a full description). Here we will get Webb filters from the SVO database.

In [12]:
from synthesizer.filters import Filter, FilterCollection

# now get the filter codes for XMM-OM
# Needs to be in angstroms (it is)
lams_xmm = np.linspace(1883.78, 2994.45, 1000) # using min and max values from wavelength given from SVO

fil_uvm2_XMM = Filter("XMM/OM.UVM2", new_lam=lams_xmm)
#fil_uvm2_XMM = Filter("XMM/OM.UVM2", new_lam=grid.lam)

# filter_codes = [f"JWST/NIRCam.{f}" for f in ["F070W", "F444W"]]
#filter_codes = ["XMM/OM.UVM2"] 
#vo_fil_xmm = FilterCollection(filter_codes=filter_codes, new_lam=lams_xmm)

# now get for UVOT
lams_uvot = np.linspace(1699.08, 2964.30, 1000) # using min and max values from wavelength given from SVO

fil_uvm2_UVOT = Filter("Swift/UVOT.UVM2", new_lam=lams_uvot)

##filter_codes = ["Swift/UVOT.UVM2",  "XMM/OM.UVM2"] 
#print(filter_codes)

#svo_fil_uvot = FilterCollection(filter_codes=filter_codes, new_lam=lams_uvot)

# make filter collection with both UVOT and OM 
#  passing the filters to the grid to limit the wavelength values.
filt_lst = [fil_uvm2_UVOT, fil_uvm2_XMM]

combined_filters = FilterCollection(
    filters=filt_lst, new_lam=lams_xmm
)

print(combined_filters)

for i in combined_filters:
    print(i)


In [11]:

photo_lums = gal.stars.particle_spectra["incident"].get_photo_luminosities(fil_uvm2_XMM)
print("Photo Luminosities: ", photo_lums)

TypeError: 'Filter' object is not iterable

In [22]:
grid.lam

unyt_array([5.5999999e+00, 6.5000000e+00, 7.4000001e+00, ..., 1.6000000e+08,
       2.4000000e+08, 3.6000000e+08], 'Å')

In [23]:
lams_xmm

array([1883.78      , 1884.89178178, 1886.00356356, 1887.11534535,
       1888.22712713, 1889.33890891, 1890.45069069, 1891.56247247,
       1892.67425425, 1893.78603604, 1894.89781782, 1896.0095996 ,
       1897.12138138, 1898.23316316, 1899.34494494, 1900.45672673,
       1901.56850851, 1902.68029029, 1903.79207207, 1904.90385385,
       1906.01563564, 1907.12741742, 1908.2391992 , 1909.35098098,
       1910.46276276, 1911.57454454, 1912.68632633, 1913.79810811,
       1914.90988989, 1916.02167167, 1917.13345345, 1918.24523524,
       1919.35701702, 1920.4687988 , 1921.58058058, 1922.69236236,
       1923.80414414, 1924.91592593, 1926.02770771, 1927.13948949,
       1928.25127127, 1929.36305305, 1930.47483483, 1931.58661662,
       1932.6983984 , 1933.81018018, 1934.92196196, 1936.03374374,
       1937.14552553, 1938.25730731, 1939.36908909, 1940.48087087,
       1941.59265265, 1942.70443443, 1943.81621622, 1944.927998  ,
       1946.03977978, 1947.15156156, 1948.26334334, 1949.37512